# 2. Kuberneties Architecture

## 2.1 컴포넌트 종류
1. Master Component : 클러스터 관리에 필수적인 Master 용 컴포넌트
2. Node Component : 일반 노드 용 
3. Addon Component : 필수는 아니며 추가로 사용할 수 있는 컴포넌트

![kubernetes_architecture](img/kubernetes_architecture.png)

- 이미지 출처 : https://cloud.kt.com/portal/user-guide/education-eduadvanced-edu_adv_3

![kubernetes_flow2](img/kubernetes_flow2.png)

<hr>


## 2.2 마스터 컴포넌트

- 일반적으로 고가용성을 위한 3중화(3대 구성 운영, 리더마스터 1, 대기2)
- **마스터 노드는 쿠버네티스의 클러스터를 관리하는 역할**을 담당 (워커노드는 에플리케이션 컨테이너용)
- 관리자만 접근할 수 있게 보안 설정을 해야 하며, 모듈의 확장성을 고려하여 기능별로 나누어져 있는 것이 특징
    - etcd, kube-apiserver, kube-scheduler, kube-controller-manager, kublet, kube-proxy 등

### 2.2.1. Kube-apiserver (API Component)
- 운영자 및 내부 노드와 통신하기 위한 인터페이스 (HTTP(S) RestAPI로 노출됨)
- 클러스터가 받는 모든 API 요청의 문법, 변환 등의 유효성 검사 및 처리하는 핵심 모듈
- 선언된 state를 key-value로 etcd 저장소에 저장하고, 저장된 상태를 조회
- 실행시 다른 컴포넌트들과 소통하면서 통신
    - ex) etcd에게 노드들의 상태 정보를 얻어오고, api가 다시 scheduler에게 전달
- 수평적으로 확장 가능하게 설계되어 서버 여러대서 운용 가능

### 2.2.2 etcd
- key-value 타입의 노드 상태 정보 저장소
- 상태정보 : H/W 리소스 어떻게 사용중인지, Container 동작 상태, Pods 관련 등
- 분산 시스템에서 노드 사이의 상태를 공유하는 consensus raftd 알고리즘으로 구현됨

### 2.2.3. kube-scheduler
- API로 부터 받은 etcd 정보로, 어떤 노드가 가장 적절한지 scheduler 판단
- 서비스를 리소스 상황에 맞게 적절한 노드에 배치하는 역할
    - ex) 자원, 하드웨어 요구사항 
    - ex) 동일 노드 파드 실행 기법 어피니트, 분산노드 실행 기법 안티 어피니티 조건 만족 여부 등
- 적합한 실행 노드의 Kublet으로 명령을 보냄

### 2.2.4. kube-controller-manager
- 파드를 관찰하며 개수를 보장해주는 역할
- Kubernetes에 있는 모든 object, 컨트롤러(복제, 배포, 상태, 크론) 등 상태를 관리
    - ex) 동작중인 Container 중 1개가 다운될시 API에게 요청 보내고 API가 Scheduler를 통해 다시 노드 할당 및 생성 
- 클러스터 안에서 새로운 컨트롤러 사용시 컨트롤러에 해당하는 구조체를 만들고, 이 구조체를 kube-controller-manager에서 관리하는 Queue에 넣어서 실행

<hr>


## 2.3. 워커 노드 (Node)
- 실제 서비스(컨테이너)가 실행되는 서버 마스터의 API Server와 통신하며 서비스를 생성하고 상태를 관리
- Kubelet : 서비스(컨테이너)를 실행/중지하고 상태를 체크하여 계속해서 살아있는 상태로 관리 
- Proxy : 네트워크 프록시와 로드 발란서 역할 (creates a iptable rule/...) 

![kubernetes_node](img/kubernetes_node.png)

<hr>


## 2.4 노드용 컴포넌트 (Node Component)
### 2.4.1. kubelet (명령 도구)
- 모든 노드에서 실행되는 k8s 에이전트로 데몬 형태 동작
- 마스터의 kube-apiserver와 통신하면서 Pod의 생성, 관리, 삭제를 담당
- 워커 노드 에서는 scheduler로 부터 받은 파드스펙(PodSpec)을 Docker로 전달하여 container 실행
- 내부적으로 cAdviser라는 모니터링 Tool을 통해 현재의 컨테이너에 대한 정보 수집, etcd에 저장

### 2.4.2. kube-proxy (공통)
- k8s의 network동작을 관리 및 ip-tables rule 구성

### 2.4.3. 컨테이너 런타임
- 컨테이너 실행 엔진으로 OCI (Open Container initiative)의 런타임 규격을 따르는 docker, containerd, runc 런타임 지원

<hr>



## 2.5. 애드온용 컴포넌트 (addon Component)
- 클러스터 안에서 특정 필요 기능을 수행 하는 파드
- namespace는 kube-system이며 애드온 파드들은 deployment, replication 컨트롤러 등으로 관리

### 2.5.1. 네트워킹 애드온
- 클러스터 안 가상 네트워크 구성시 kube-proxy 외에 네트워킹 에드온을 사용

### 2.5.2. DNS 애드온
- 클러스터 안에서 동작하는 DNS 서버로 쿠버네티스 서비스에 DNS 레코드를 제공
- 쿠버네티스 안에 실행된 컨테이너들은 자동으로 DNS 서버에 등록됨
- 종류 : CoreDNS, Kube-dns
- 초기에는 kube-DNS가 많이 사용, 버그가 많아서 나중에는 CoreDNS를 기본 애드온으로도 많이 사용

### 2.5.3. 기타
- 대시보드 애드원, 컨테이너 자원 모니터링(cAdvisor), 
- 클러스터 로깅(ELK, EFK, DataDog)
    - k8s운영 로그들을 중앙화

<hr>

## 2.6. MasterNode - WorkerNode

![kubernetes_component](img/kubernetes_component.png)

- 이미지 출처 : https://cloud.kt.com/portal/user-guide/education-eduadvanced-edu_adv_3
